# Social Influence Project - Reddit Submission Popularity RCT <a class="anchor" id="first-bullet"></a>

This notebook concretely illustrates the data gathering and analysis for which the main paper is based upon.

# Table of contents:
TODO

# Data gathering setup and pipeline

The following section contains all the relevant python code used to interface with reddit and store the resulting data in a local SQLite database. Keep in mind that the code is not meant to be deployed and run from within a jupyter notebook. Instead, it was designed such that the entrypoint `main.py` can be run on a schedule with a cronjob. For a greater viewing experience and better overview, it is recommended to visit [the GitHub repo](https://github.com/NValsted/RDS-Project-2022-1)

### Database interface

In [1]:
#src/database.py

from dataclasses import dataclass
from typing import Optional, TypeVar, List, Type
from contextlib import contextmanager

from sqlalchemy.engine import Engine
from sqlalchemy.sql.schema import Table
from sqlmodel import create_engine, SQLModel, Session

ModelType = TypeVar("ModelType", bound=SQLModel)


@dataclass
class Database:
    """
    Database class with methods to create/drop tables and add/retrieve table entries
    """
    engine: Engine

    @contextmanager
    def session(self):
        with Session(self.engine) as session:
            yield session

    def create_tables(self, tables: Optional[List[Table]] = None) -> None:
        SQLModel.metadata.create_all(self.engine, tables=tables)

    def drop_tables(self, tables: Optional[List[Table]] = None) -> None:
        SQLModel.metadata.drop_all(self.engine, tables=tables)

    def add(self, instances: List[ModelType]) -> None:
        with self.session() as session:
            session.add_all(instances)
            session.commit()

    def get(self, model: Type[ModelType], id: int) -> Optional[ModelType]:
        with Session(self.engine) as session:
            matches = session.query(model).filter(model.id == id).all()
            if len(matches) > 1:
                raise ValueError(
                    f"Multiple matches for {id=} in {model.__name__}:\n{matches}"
                )
            elif len(matches) == 0:
                return None
            return matches[0]


@dataclass
class DBFactory:
    """
    Factory to create Database instances
    """
    engine_url: str = "sqlite:///../database.db"

    def __call__(self, *args, **kwargs) -> Database:
        engine = create_engine(url=self.engine_url, **kwargs)
        return Database(engine=engine, **kwargs)

### Model and database table definitions
Python Pydantic models and SQLite table definitions are made simultaneously using the SQLModel ORM capabilities

A `RedditPost` entry will be created once when a post is fetched the first time, which includes generic metadata about the post, while a `RedditPostLogPoint` will be created periodically, which is responsible for keeping track of the score and number of comments. 

In [2]:
# src/database_models.py
from enum import Enum
from datetime import datetime
from typing import Optional

from sqlalchemy import Column, Enum as SAEnum
from sqlmodel import SQLModel, Field


class GroupEnum(str, Enum):
    CONTROL = "CONTROL"
    TREATMENT = "TREATMENT"


class RedditPost(SQLModel):
    id: str = Field(primary_key=True, index=True)
    batch_id: str = Field(
        index=True,
        description="Unique ID of the batch in which the post was added",
    )
    active: bool = Field(
        default=True, description="Indicates whether the post is reachable"
    )
    group: GroupEnum = Field(sa_column=Column(SAEnum(GroupEnum)))
    subreddit: str = Field()
    title: str = Field()
    creation_date: datetime = Field(description="Date at which post was created")


class RedditPostTable(RedditPost, table=True):
    __tablename__ = "RedditPost"


class RedditPostLogPoint(SQLModel):
    pk: Optional[int] = Field(primary_key=True, default=None, index=True)
    id: str = Field(index=True)
    score: int = Field()
    num_comments: int = Field()
    date: datetime = Field(description="Date at which stats were collected")


class RedditPostLogPointTable(RedditPostLogPoint, table=True):
    __tablename__ = "RedditPostLogPoint"

### Utilities
A few utility functions are also defined which are primarily concerned with increasing the robustness of the data gathering solution.

The logger and its factory method provide a structured interface for saving logs persistently to disk when the main job runs for longer periods of time without manual intervention, and the `safe_call` will prevent the process from terminating immediately on any error - e.g. if a single post out of thousands in a batch raise a 404 error because it was deleted, then we still want to process the rest of the batch. Likewise, it makes sense to retry requests if the connection temporarily drops.

In [3]:
# src/utils.py
from typing import Callable, Any, List, Dict, Optional
from time import sleep
import logging
import traceback

from prawcore import exceptions


def get_logger(name: str = "RDS-PROJECT") -> logging.Logger:
    logger = logging.getLogger(name)
    fhandler = logging.FileHandler(filename="logs.log", mode="a")
    formatter = logging.Formatter(
        "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    )
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)
    return logger


def safe_call(
    func: Callable,
    args: Optional[List] = None,
    kwargs: Optional[Dict] = None,
    max_retries: int = 3,
    sleep_time: int = 1,
    exception: Exception = exceptions.NotFound,
    raise_on_failure: bool = True,
) -> Any:
    """
    Wraps a function and retries it if it raises an exception.
    """
    logger = get_logger()

    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    error = Exception("Unknown error")

    while max_retries > 0:
        try:
            return func(*args, **kwargs)
        except exception as e:
            error = e
            max_retries -= 1
            logger.info(
                f"{func.__name__} failed with args {args} and kwargs {kwargs}\n"
                f"{e}\n{traceback.format_exc()}"
                f"{max_retries} retries left"
            )
            sleep(sleep_time)

    if raise_on_failure:
        logger.error(f"Failed to execute function {func.__name__}")
        raise error

### The reddit interface
Interfacing with reddit is done via the praw reddit API wrapper, which in turn is wrapped in the RedditBot class. This class provides methods to:
- Fetch new posts:
    - A random batch of new posts can be fetched with `get_batch_of_posts`
    - This batch of posts can be randomly divided into CONTROL and TREATMENT groups with `group_posts`
- Interface with database:
    - Add:
        - Posts (i.e. metadata about subreddit, creation_date, CONTROL/TREATMENT group, etc.) can be added with `add_posts_to_db`
        - Log points (i.e. observations of score and number of comments) can be added with `add_log_points`
    - Get:
        - Posts no older than a certain amount of days can be retrieved from the database with `get_stored_posts`
- The data (title, score, etc.) for a list of posts can be fetched given a list of ids with `get_posts`

In [4]:
# src/reddit_bot.py
import os
import random
import traceback
from datetime import datetime, timedelta
from multiprocessing.pool import ThreadPool
from typing import Tuple, List, Optional, Dict
from uuid import uuid4
import json

import praw

# Local imports suppressed in notebook cells since the objects are already available in scope.
# from src.database import DBFactory
# from src.database_models import RedditPostTable, RedditPostLogPointTable, GroupEnum
# from src.utils import get_logger, safe_call

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
USER_AGENT = os.getenv("USER_AGENT")
RATELIMIT = int(os.getenv("RATELIMIT", 5))

logger = get_logger("REDDIT-BOT")


class RedditBot:
    """
    Wrapper for the Reddit bot.

    It contains the following methods:
    - get_batch_of_posts: Selects a batch of posts for the experiment
    - group_posts: Groups a list of posts into treatment and control groups
    - add_posts_to_db: Adds a list of posts to the database
    - add_log_points: Adds a list of log points to the database
    - get_stored_posts: Fetches posts from the database given a date filter
    - get_posts: Fetches posts from the Reddit API given a list of ids
    """

    reddit: praw.Reddit
    url: str = "https://www.reddit.com"

    def __init__(
        self,
        client_id: str = CLIENT_ID,
        client_secret: str = CLIENT_SECRET,
        username: str = USERNAME,
        password: str = PASSWORD,
        user_agent: str = USER_AGENT,
        ratelimit: int = RATELIMIT,
    ):
        """
        Authenticates the bot and initializes the Reddit instance.
        """
        assert isinstance(client_id, str)
        assert isinstance(client_secret, str)
        assert isinstance(username, str)
        assert isinstance(password, str)
        assert isinstance(user_agent, str)
        assert isinstance(ratelimit, int)

        self.reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            username=username,
            password=password,
            user_agent=user_agent,
            ratelimit=ratelimit,
        )

    def get_batch_of_posts(
        self,
        subreddit: str = "all",
        score: int = 1,
        num_comments: int = 1,
        batch_size: int = 64,
    ):
        """
        Selects a batch of posts with at most 'score' number of upvotes and
        'num_comments' number of comments in the given subreddit.

        NOTE: batch_size is an upper bound on the number of posts returned.
        """

        posts = [
            post
            for post in self.reddit.subreddit(subreddit).new(limit=batch_size)
            if post.score <= score and post.num_comments <= num_comments
        ]

        return posts

    @staticmethod
    def group_posts(
        posts: List[praw.models.Submission],
    ) -> Tuple[List[praw.models.Submission], List[praw.models.Submission]]:
        """
        Assigns posts into treatment and control groups.
        """

        batch_id = str(uuid4())
        random.shuffle(posts)
        if len(posts) % 2 != 0:
            posts.pop()  # Drop a random post to make the list even

        middle = len(posts) // 2
        treatment_posts = posts[:middle]
        control_posts = posts[middle:]

        for post in treatment_posts:
            post.upvote()
            post.group = GroupEnum.TREATMENT
            post.batch_id = batch_id

        for post in control_posts:
            post.group = GroupEnum.CONTROL
            post.batch_id = batch_id

        return treatment_posts, control_posts

    @staticmethod
    def add_posts_to_db(
        posts: List[praw.models.Submission],
        backup: bool = False,
    ) -> None:

        prepared_posts = []

        def _prepare_post(post: praw.models.Submission) -> Dict:
            return dict(
                id=post.id,
                batch_id=post.batch_id,
                group=post.group,
                subreddit=str(post.subreddit),
                title=post.title,
                creation_date=post.created_utc,
            )

        for post in posts:
            prepared_post = safe_call(
                _prepare_post,
                args=[post],
                exception=Exception,
                raise_on_failure=False,
            )
            if prepared_post is not None:
                prepared_posts.append(prepared_post)

        if backup:
            today = datetime.today().date().isoformat()
            with open(f"backup/REDDITBOT_{today}_{str(uuid4())}.json", "w") as f:
                json.dump(prepared_posts, f, indent=4, default=str)

        db = DBFactory()()

        parsed_posts = [RedditPostTable(**post) for post in prepared_posts]
        db.add(parsed_posts)
        logger.info(f"Added {len(parsed_posts)} posts to the database")

        RedditBot.add_log_points(posts, backup=backup)

    @staticmethod
    def add_log_points(
        posts: List[praw.models.Submission], backup: bool = False
    ) -> None:

        prepared_posts = []
        stale_posts = []

        def _prepare_post(post: praw.models.Submission) -> Dict:
            return dict(
                id=post.id,
                score=post.score,
                num_comments=post.num_comments,
                date=datetime.now(),
            )

        for post in posts:
            prepared_post = safe_call(
                _prepare_post,
                args=[post],
                exception=Exception,
                raise_on_failure=False,
            )
            if prepared_post is not None:
                prepared_posts.append(prepared_post)
            else:
                stale_posts.append(post)

        if backup:
            today = datetime.today().date().isoformat()
            with open(f"backup/REDDITBOT_{today}_{str(uuid4())}.json", "w") as f:
                json.dump(prepared_posts, f, indent=4, default=str)

        db = DBFactory()()

        parsed_posts = [RedditPostLogPointTable(**post) for post in prepared_posts]
        db.add(parsed_posts)
        logger.info(f"Added {len(parsed_posts)} log points to database")

        for post in stale_posts:
            old_instance = db.get(RedditPostTable, id=post.id)
            if old_instance is not None:
                old_instance.active = False
                db.add([old_instance])

        logger.info(f"Marked {len(stale_posts)} stale posts")

    @staticmethod
    def get_stored_posts(max_age: int = 8) -> List[RedditPostTable]:
        db = DBFactory()()

        with db.session() as session:
            posts = (
                session.query(RedditPostTable)
                .filter(
                    RedditPostTable.creation_date
                    >= (datetime.now() - timedelta(days=max_age))
                )
                .filter(RedditPostTable.active)
                .all()
            )

            logger.info(f"Fetched {len(posts)} active posts from the database")

            return posts

    def _submission_wrapper(self, *args, **kwargs) -> Optional[praw.models.Submission]:
        try:
            return self.reddit.submission(*args, **kwargs)
        except Exception as e:
            logger.error(
                f"{e}\n{traceback.format_exc()}\nargs: {args}\nkwargs: {kwargs}"
            )
            return None

    def get_posts(
        self, ids: List[str], threads: int = 4
    ) -> List[praw.models.Submission]:
        with ThreadPool(threads) as pool:
            posts = pool.map(self._submission_wrapper, ids)

        posts = [post for post in posts if post is not None]

        return posts

### One-time entrypoint - `setup.py`
The `setup` function simply establishes a connection to- and possibly creates the database, after which it drops any existing tables and creates new ones afresh.

In [5]:
# setup.py

# Local imports suppressed in notebook cells since the objects are already available in scope.
# from src.database_models import RedditPostTable, RedditPostLogPointTable  # NOQA : F401
# from src.database import DBFactory


def setup():
    db = DBFactory()()
    db.drop_tables()
    db.create_tables()

### Main entrypoint - `main.py`
With everything set up, the `main` function defines a simple routine for monitoring the stats of previously fetched posts as well as fetching a batch of newly created posts. This is the function that is deployed to run periodically. Check [the GitHub repo](https://github.com/NValsted/RDS-Project-2022-1) for more details.

In [6]:
# main.py

# Local imports suppressed in notebook cells since the objects are already available in scope.
# from src import RedditBot
# from src.utils import safe_call


def main():
    bot = RedditBot()

    # Update old Posts
    posts = bot.get_stored_posts()
    ids = {post.id for post in posts}

    posts = bot.get_posts(ids)
    bot.add_log_points(posts)

    # New posts
    treatment, control = safe_call(
        func=lambda: bot.group_posts(bot.get_batch_of_posts())
    )
    bot.add_posts_to_db(treatment)
    bot.add_posts_to_db(control)

# Data analysis
At this point, roughly 40000 posts have been fetched and monitored over the course of 7 days each, which has resulted in around 2.5 million log points. This section marks the start of an analysis of the resulting data.

A snapshot of the database is available at https://ituniversity-my.sharepoint.com/:u:/g/personal/nicv_itu_dk/EUPPi364WC1Am9hZAJ__3ScByk-dMOuZBrM0VteAbQjhYw?e=lkJ3Q2

In [7]:
from datetime import timedelta

import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

## Load data

In [8]:
reddit_post_df = pd.read_sql_table(RedditPostTable.__tablename__, DBFactory.engine_url)
reddit_post_log_point_df = pd.read_sql_table(RedditPostLogPointTable.__tablename__, DBFactory.engine_url)

## Preprocessing

Before commencing with the analysis, a little preprocessing is beneficial, e.g. due to the fact that certain posts are marked inactive since they have been deleted or otherwise made unreachable, which has unbalanced the dataset slightly.

The preprocessing steps are the following (Which are intertwined in practice for convenience):
- Join post info and log points (`RedditPostTable` and `RedditPostLogPointTable`)
- Balance dataset. 
    - For each inactive post, identify a post from the conjugate group with the same `batch_id` and filter away both.
    - Filter away any post that has not been monitored for at least 7 days (i.e. younger than 7 days or marked inactive before the 7 day mark).
- Create derived columns: `age` and `saturation`.
- Unbias treatment group by subtracting 1 from the score
- For each of the two groups, create dataframes containing only the latest log point for a post.

These steps will be described further in the relevant cells.

### Join dataframes
A join between the RedditPost and RedditPostLogPoint on the `id` column is done (reddit's id for a given post), which essentially yields RedditPostLogPoint but with all the relevant metadata attached for the post which the log point corresponds to. A random sample of the dataframe is shown as an example of the resulting format.

In [9]:
joined = pd.merge(reddit_post_df, reddit_post_log_point_df, on="id", how="left")

In [10]:
joined.sample(n=5)

,group,id,batch_id,active,subreddit,title,creation_date,pk,score,num_comments,date
1758912,CONTROL,u7kha3,58e477e3-afd6-40c8-9ec5-541c02e697b1,True,kucoin,Most Crypto friendly country,2022-04-20 01:05:16,1887674,2,10,2022-04-28 00:32:10.161076
855411,CONTROL,tjzl5k,45f07b5f-e7e8-448c-b5ec-58b1436b7471,True,granturismo,"Wow, they really did make the car prices reali...",2022-03-22 10:06:22,765566,731,61,2022-03-23 15:39:51.274643
846567,CONTROL,tjugzb,e038c323-06b2-4337-8dba-c32a70d8558d,True,Amillennialism,lol wut,2022-03-22 04:06:17,907084,4,1,2022-03-28 00:24:20.887211
2031129,CONTROL,udrkme,d2441cd4-a212-4d40-8286-7f3cbe58d9ca,True,CryptoHatcher,Using Binance,2022-04-28 10:06:08,1998821,1,0,2022-05-01 12:15:33.878562
454359,TREATMENT,taqe69,3b0a1c05-2687-461e-88f3-01cab341c760,True,TeamfightTactics,Who needs 3 Magnetic Removals??,2022-03-10 04:24:14,259732,17,9,2022-03-10 19:20:12.294425


### Derived columns p0 - age
The `age` column simply denotes how long it has been since a post was created when a log point was recorded.

In [11]:
joined["age"] = joined["date"] - joined["creation_date"]

### Unbias TREATMENT group
The initial upvote of 1 is subtracted from the TREATMENT group, since we are interested in investigating whether the treatment has increased popularity as expressed by external users - i.e. all users that are not our bot.

Perhaps, this is best explained with an example in the extremes: If we assume an artificial world where no other users interact with posts on the platform, then we can safely say beforehand that the treatment will not have an effect. If we then perform our experiment, all posts in the CONTROL group will have a score of 0, and all posts in the TREATMENT group will have a score of 1. If we were to perform any analysis on the resulting data without unbiasing the TREATMENT group, we would wrongfully conclude effectiveness of the TREATMENT due to the difference in score distributions. 

In [12]:
joined.loc[joined["group"] == GroupEnum.TREATMENT.value, ("score")] -= 1

### Latest posts
We have record many log points for each individual post, so the `joined_latest` dataframe consists of only the latest log point for each post  

In [13]:
joined_latest = joined.sort_values(["date"], ascending=False).groupby(by="id", as_index=False).first()

### Balance dataset
To balance the dataset, we want remove any posts that have not yet been tracked for at least 7 days, and we also want to ensure that there are equally many data points in the control and treatment groups. Since we sample an equal amount of control and treatment posts in each batch, the only reason these can be different is if one or more posts have been marked inactive (i.e. deleted or otherwise unreachable)

In [14]:
ids_to_drop = set()

First, filter posts younger than 7 days:

In [15]:
for idx, row in joined_latest[joined_latest["age"] < timedelta(days=7)].iterrows():
    ids_to_drop.add(str(row["id"]))

Then balance pairs within batches, prioritizing removing conjugate posts which are also inactive - e.g. if a batch contains a single inactive control post and inactive treatment post, then these two should simply cancel out. Otherwise, simply sample a random active post of the conjugate group to cancel out. 

In [16]:
inactive_posts = joined_latest[joined_latest["active"] == False]

for group in inactive_posts.groupby(by="batch_id", as_index=False):
    group_key, group_df = group

    control_remainder, treatment_remainder = (
        joined_latest[
            (joined_latest["batch_id"] == group_key)
            & (joined_latest["group"] == group.value)
            & ~(joined_latest["id"].isin(set(group_df["id"])))
        ]
        for group in (GroupEnum.CONTROL, GroupEnum.TREATMENT)
    )
    num_control = control_remainder.shape[0]
    num_treatment = treatment_remainder.shape[0]
    
    if num_control != num_treatment:
        conjugate_remainder = control_remainder if num_treatment < num_control else treatment_remainder
        num_to_drop = abs(num_control - num_treatment)
        to_drop = conjugate_remainder.sample(n=num_to_drop)
        ids_to_drop.add(str(to_drop["id"]))
        
    for entry in group_df["id"]:
        ids_to_drop.add(str(entry))

Apply filters

In [17]:
print(f"Dropping {len(ids_to_drop)} post(s)")

joined_latest = joined_latest[~(joined_latest["id"].isin(ids_to_drop))]
# joined = joined[~(joined["id"].isin(ids_to_drop))]

Dropping 4335 post(s)


### Derived columns p1 - saturation
Saturation is useful in a temporal context and describes the ratio of some maximum value for a post. E.g. a post with `score` values 0, 50, and 100 will be mapped to `score_saturation` values of 0, 0.5, 1.0.

In [18]:
joined["score_saturation"] = joined["score"] / joined.groupby("id")["score"].transform(np.max)
joined["num_comments_saturation"] = joined["num_comments"] / joined.groupby("id")["num_comments"].transform(np.max)

### Overview of resulting dataframe

In [19]:
joined.sample(n=5)

,group,id,batch_id,active,subreddit,title,creation_date,pk,score,num_comments,date,age,score_saturation,num_comments_saturation
1467909,CONTROL,u0s8et,1bc51938-e0e3-4e64-809c-0359e152a540,True,pokemongo,crazy community day here with the new incense ...,2022-04-10 22:03:08,1462574,12,7,2022-04-14 18:14:22.385070,3 days 20:11:14.385070,0.857143,1.0
593025,TREATMENT,tdumdy,d83ebc33-8e43-42af-b9ef-a3a79061cc52,True,SoundCloudHipHop,[FRESH] KreamDaReap - ReapInTheFlesh (Prod.Xantu),2022-03-14 10:43:42,630288,2,1,2022-03-19 12:25:19.106356,5 days 01:41:37.106356,1.000000,1.0
546293,CONTROL,tcsw69,76c78e4e-702f-46d6-8951-db40397da591,True,DMAcademy,"How to run wilderness ""chase"" (party being tra...",2022-03-12 22:46:51,607401,1,5,2022-03-18 18:50:57.080665,5 days 20:04:06.080665,1.000000,1.0
870640,CONTROL,tkei94,a563bfdb-8464-484a-975a-0132136fffbe,True,theisle,Tenno what the hell dood?,2022-03-22 22:07:23,803955,2,2,2022-03-24 21:02:08.858438,1 days 22:54:45.858438,1.000000,1.0
734283,CONTROL,th4tk1,8fbc5db6-87db-4636-bc62-2f8c64afded8,True,SeikoMods,1.7mm Allan Key Needed,2022-03-18 16:06:21,683888,1,11,2022-03-21 03:35:17.046362,2 days 11:28:56.046362,1.000000,1.0


### Split dataframe in groups
Finally, we simply create dataframes containing only values from the respective groups for convenience.

In [20]:
control, treatment = (joined[joined["group"] == group.value] for group in (GroupEnum.CONTROL, GroupEnum.TREATMENT))
control_latest, treatment_latest = (
    joined_latest[joined_latest["group"] == group.value] for group in (GroupEnum.CONTROL, GroupEnum.TREATMENT)
)

## Distribution similarity
The following section investigates the similarity between the score and number of comments distributions between the two groups, in order to evaluate the effectiveness of the treatment.

The following plot displays histograms of the data points within each group. The general distribution type seems similar, but parameters seem to be noticeably different.

In [21]:
from plotly import io as pio, graph_objects as go, express as px
from plotly.subplots import make_subplots
pio.renderers.default = "iframe"
# Jupyter notebooks don't handle plotly figures well.
# Therefore, iframes and %%capture are used to save the resulting html files to disk instead.
# These should appear in the iframe_figures directory, following the figure_{cell_number}.html naming scheme.

In [22]:
%%capture

fig = make_subplots(
    rows=2,
    cols=2,
    shared_yaxes=True,
    subplot_titles=("Score distribution", "Number of comments distribution"),
    horizontal_spacing=0.05,
)

for i in (1, 2):
    for j, attr in ((1, "score"), (2, "num_comments")):
        for df, color in ((control_latest, "#4969AA"), (treatment_latest, "#C90D0D")):
            fig.add_trace(
                go.Histogram(
                    x=df[attr],
                    histnorm="percent",
                    name=df["group"].min(),
                    xbins=dict(
                        start=df[attr].min(),
                        end=df[attr].max(),
                        size=0.5
                    ),
                    marker_color=color,
                    opacity=0.75,
                    showlegend=i == 1 and j == 1
                ),
                row=i,
                col=j,
            )

        fig.update_layout(
            yaxis_title_text="Percent",
            bargap=0.2,
            bargroupgap=0.1
        )

fig.update_xaxes(type="log", row=1, col=1)
fig.update_xaxes(type="log", row=1, col=2)
fig.update_xaxes(title_text="Score (log)", row=2, col=1, type="log")
fig.update_xaxes(title_text="Number of comments (log)", row=2, col=2, type="log")

fig.update_yaxes(title_text="Percent (log)", type="log", row=2, col=1)
fig.update_yaxes(type="log", row=2, col=2)

fig.show()

### Kolmogorov-Smirnov test
With the Kolmogorov-Smirnov test, we perform an empirical test to investigate whether the data points appear to belong to the same distribution. The null hypothesis is that the distributions are identical, and thus a low p-value indicates that there is a significant difference between the control and treatment groups. As can be seen below, the results are in favour of the alternative hypothesis.

In [23]:
stats.kstest(control_latest["score"], treatment_latest["score"])

KstestResult(statistic=0.050203606748109364, pvalue=3.0660866043136154e-19)

In [24]:
stats.kstest(control_latest["num_comments"], treatment_latest["num_comments"])

KstestResult(statistic=0.03378460953297813, pvalue=6.0263840562854085e-09)

However, small fluctuations might provide an inaccurate picture - e.g. a post with a score of 34 is probably not significantly different from one with a score of 35. So we will investigate whether rounding the score and number of comments to the nearest multiple of a range of numbers will jeopardize the result above.

In [25]:
binned_ks_test_score = [stats.kstest(*(df["score"].map(lambda x: i * round(x / i)) for df in (control_latest, treatment_latest))).pvalue for i in tqdm(range(1, 10))]
binned_ks_test_num_comments = [stats.kstest(*(df["num_comments"].map(lambda x: i * round(x / i)) for df in (control_latest, treatment_latest))).pvalue for i in tqdm(range(1, 10))]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 22.14it/s]


...But as the plot below shows, even pessimistically trying several different bin sizes, the maximum p-value is still no larger than on the order of $10^{-8}$, which occurs for the score distributions when rounding the score to the nearest multiple of 5. Thus, it seems more likely that there is a significant difference between the control and treatment groups. 

In [26]:
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=("KS-test binned score distributions", "KS-test binned number of comments distributions"),
)
for i, lst in enumerate((binned_ks_test_score, binned_ks_test_num_comments)):
    fig.add_trace(go.Scatter(x=list(range(1, len(lst) + 1)), y=lst), row=i+1, col=1)
fig.update_yaxes(type="log", row="all", col="all")
fig.update_layout(showlegend=False)
fig.show()

### Descriptive statistics
The following section simply displays key descriptive statistics for the 2 distribution pairs

#### Quantile statistics

In [27]:
for attr in ("score", "num_comments"):
    for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest)):
        print(f"{name}: {attr}")
        print(df[attr].describe())
        print()

CONTROL: score
count    17142.000000
mean        73.678684
std        757.802228
min          0.000000
25%          1.000000
50%          2.000000
75%         14.000000
max      43660.000000
Name: score, dtype: float64

TREATMENT: score
count    17190.000000
mean        80.589412
std        939.957942
min         -1.000000
25%          1.000000
50%          3.000000
75%         16.000000
max      52543.000000
Name: score, dtype: float64

CONTROL: num_comments
count    17142.000000
mean        12.400187
std        263.191000
min          0.000000
25%          0.000000
50%          2.000000
75%          7.000000
max      32770.000000
Name: num_comments, dtype: float64

TREATMENT: num_comments
count    17190.000000
mean        11.706923
std         76.597475
min          0.000000
25%          0.000000
50%          2.000000
75%          8.000000
max       6577.000000
Name: num_comments, dtype: float64



#### Skewness and Kurtosis

In [28]:
for metric in ("skew", "kurtosis"):
    print(f"METRIC: {metric}")
    print(
        "\n".join(
            (
                f"{name} {attr} {metric}: {getattr(df[attr], metric)()}"
                for attr in ("score", "num_comments")
                for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest))
            )
        )
    )
    print()

METRIC: skew
CONTROL score skew: 31.450308607514575
TREATMENT score skew: 38.79248590691322
CONTROL num_comments skew: 114.51770761538164
TREATMENT num_comments skew: 53.29467656908218

METRIC: kurtosis
CONTROL score kurtosis: 1311.6424276146558
TREATMENT score kurtosis: 1865.065871456808
CONTROL num_comments kurtosis: 14058.14667758193
TREATMENT num_comments kurtosis: 3912.1044260950575



#### (score, num_comments)-correlation

In [29]:
for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest)):
    print(f"{name}:")
    print(df["score"].corr(df["num_comments"]))
    print()

CONTROL:
0.09335078773647593

TREATMENT:
0.15700463173949747



#### Filter extremes

In [30]:
def filter_top(df: pd.DataFrame, attr: str, qt: float = 0.95):
    return df[df[attr] <= df[attr].quantile(qt)][attr]

In [31]:
for attr in ("score", "num_comments"):
    for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest)):
        print(f"{name}: {attr}")
        print(df[df[attr] <= df[attr].quantile(0.95)][attr].describe())
        print()

CONTROL: score
count    16286.000000
mean        13.329915
std         28.073410
min          0.000000
25%          1.000000
50%          2.000000
75%         10.000000
max        183.000000
Name: score, dtype: float64

TREATMENT: score
count    16331.000000
mean        14.657216
std         30.408935
min         -1.000000
25%          1.000000
50%          2.000000
75%         12.000000
max        201.000000
Name: score, dtype: float64

CONTROL: num_comments
count    16301.000000
mean         4.550028
std          6.644784
min          0.000000
25%          0.000000
50%          2.000000
75%          6.000000
max         35.000000
Name: num_comments, dtype: float64

TREATMENT: num_comments
count    16348.000000
mean         5.264497
std          7.666882
min          0.000000
25%          0.000000
50%          2.000000
75%          7.000000
max         41.000000
Name: num_comments, dtype: float64



In [32]:
for metric in ("skew", "kurtosis"):
    print(f"METRIC: {metric}")
    print(
        "\n".join(
            (
                f"{name} {attr} {metric}: {getattr(df[df[attr] <= df[attr].quantile(0.95)][attr], metric)()}"
                for attr in ("score", "num_comments")
                for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest))
            )
        )
    )
    print()

METRIC: skew
CONTROL score skew: 3.368455633673462
TREATMENT score skew: 3.3582937899142324
CONTROL num_comments skew: 2.114025515346434
TREATMENT num_comments skew: 2.165640646130761

METRIC: kurtosis
CONTROL score kurtosis: 12.296269499843739
TREATMENT score kurtosis: 12.336831661854085
CONTROL num_comments kurtosis: 4.539776376481717
TREATMENT num_comments kurtosis: 4.839831307073403



In [33]:
for name, df in (("CONTROL", control_latest), ("TREATMENT", treatment_latest)):
    print(f"{name}:")
    print(filter_top(df, "score").corr(filter_top(df, "num_comments")))
    print()

CONTROL:
0.2924712999654716

TREATMENT:
0.28505258035361164



## Temporal similarity
TODO

In [34]:
%%capture
px.scatter(joined, x="age", y="score_saturation", color="group", opacity=0.005)

In [35]:
control["score_saturation"].mean()

0.8195551703033446

In [36]:
treatment["score_saturation"].mean()

-inf

# Draft conclusion

In conclusion: From the KS-tests we deduce that the probability of the samples from the two distribution pairs - (TREATMENT_score, CONTROL_score) and (TREATMENT_num_comments, CONTROL_num_comments) - originating from the respective same distribution are both <i>significantly</i> less than 5% (p-value < 0.05). Investigating descriptive statistics of these distributions leads us to believe that the popularity of a post in terms of its score is indeed higher for posts in the treatment group. However, the opposite is true for the number of comments. Initially, we had hypothesized that score and number of comments were correlated, which does not seem to be the case, and this does tie together with the aforementioned uncorrelated effectiveness of the treatment on score and comments. However, a word of caution: The distributions are severely heavy-tailed as indicated by the kurtosis measures, and thus the extreme sample values can easily impact the results significantly, given the fairly limited size of the sample. This is demonstrated by filtering away the top 5% of the data from each distribution, yielding noticeably different perceived relationships between the distributions.